# Convolutional Neural Network

### Importing the libraries

In [1]:
!pip install --upgrade tensorflow keras Pillow

Defaulting to user installation because normal site-packages is not writeable
Requirement already up-to-date: tensorflow in /home/renan/.local/lib/python3.6/site-packages (2.3.1)
Requirement already up-to-date: keras in /home/renan/.local/lib/python3.6/site-packages (2.4.3)
Requirement already up-to-date: Pillow in /usr/local/lib/python3.6/dist-packages (7.2.0)
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [3]:
tf.__version__

'2.3.1'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [4]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [5]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [12]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [13]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='tanh', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [14]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=4, strides=4))

### Adding a second convolutional layer

In [15]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='tanh'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a third convolutional layer

In [16]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='tanh'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=1, strides=1))

### Step 3 - Flattening

In [17]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [18]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [19]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [20]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [21]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 15)

Epoch 1/15
250/250 [==============================] - 56s 223ms/step - loss: 0.6590 - accuracy: 0.5997 - val_loss: 0.6215 - val_accuracy: 0.6645
Epoch 2/15
250/250 [==============================] - 55s 221ms/step - loss: 0.6022 - accuracy: 0.6706 - val_loss: 0.5614 - val_accuracy: 0.7125
Epoch 3/15
250/250 [==============================] - 56s 222ms/step - loss: 0.5482 - accuracy: 0.7209 - val_loss: 0.5569 - val_accuracy: 0.7165
Epoch 4/15
250/250 [==============================] - 55s 221ms/step - loss: 0.5299 - accuracy: 0.7389 - val_loss: 0.5360 - val_accuracy: 0.7290
Epoch 5/15
250/250 [==============================] - 56s 226ms/step - loss: 0.5010 - accuracy: 0.7544 - val_loss: 0.5260 - val_accuracy: 0.7465
Epoch 6/15
250/250 [==============================] - 57s 227ms/step - loss: 0.4828 - accuracy: 0.7664 - val_loss: 0.5027 - val_accuracy: 0.7600
Epoch 7/15
250/250 [==============================] - 55s 221ms/step - loss: 0.4655 - accuracy: 0.7781 - val_loss: 0.5429 - val_ac

## Part 4 - Making a single prediction

In [22]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_2.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

In [23]:
print(prediction)

cat


## Fim